In [1]:
# example_meta_data_reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [2]:
%matplotlib qt
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import h5py


In [3]:
example_set = ak.from_parquet('./output/custom_1337_meta_data.parquet')

In [4]:
example_set.fields

['event_id', 'mc_truth', 'primary_lepton_1', 'primary_hadron_1', 'total']

In [5]:
example_set.mc_truth.fields

['injection_energy',
 'injection_type',
 'injection_interaction_type',
 'injection_zenith',
 'injection_azimuth',
 'injection_bjorkenx',
 'injection_bjorkeny',
 'injection_position_x',
 'injection_position_y',
 'injection_position_z',
 'injection_column_depth',
 'primary_lepton_1_type',
 'primary_hadron_1_type',
 'primary_lepton_1_position_x',
 'primary_lepton_1_position_y',
 'primary_lepton_1_position_z',
 'primary_hadron_1_position_x',
 'primary_hadron_1_position_y',
 'primary_hadron_1_position_z',
 'primary_lepton_1_direction_theta',
 'primary_lepton_1_direction_phi',
 'primary_hadron_1_direction_theta',
 'primary_hadron_1_direction_phi',
 'primary_lepton_1_energy',
 'primary_hadron_1_energy',
 'total_energy']

In [6]:
example_set.total.fields

['sensor_id',
 'sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'sensor_string_id',
 't']

In [7]:
for field in example_set.mc_truth.fields:
    print(field)
    print(example_set.mc_truth[field])

injection_energy
[1e+04, 1e+04, 1e+04, 1e+04, 1e+04, 1e+04, 1e+04, 1e+04, 1e+04, 1e+04]
injection_type
[14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
injection_interaction_type
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
injection_zenith
[2.35, 1.61, 2.29, 1.79, 1.44, 1.64, 2.49, 1.76, 2.21, 2.44]
injection_azimuth
[4.93, 1.39, 3.37, 3.15, 5.68, 2.05, 4.54, 4.89, 5.59, 3.22]
injection_bjorkenx
[0.202, 0.531, 0.224, 0.378, 0.535, 0.0629, 0.114, 0.0101, 0.0291, 0.126]
injection_bjorkeny
[0.202, 0.531, 0.224, 0.378, 0.535, 0.0629, 0.114, 0.0101, 0.0291, 0.126]
injection_position_x
[-80.5, -37.8, -85.8, -25.1, 124, -46.6, 110, -127, -66.7, 58.6]
injection_position_y
[100, -104, -89.8, 87.5, 47, -96.3, 30.1, -4.83, 96.6, -24.9]
injection_position_z
[111, 171, 76.2, 129, -49.4, 127, -8.17, -84.1, 44.6, -137]
injection_column_depth
[3.61e+04, 2.75e+04, 2.1e+04, 2.3e+04, ... 3.14e+04, 1.53e+04, 3.37e+04, 2.06e+04]
primary_lepton_1_type
[13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
primary_hadron_1_type
[-2000001006, 

In [8]:
example_set.primary_hadron_1.fields

['sensor_id',
 'sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'sensor_string_id',
 't',
 'loss_pos_x',
 'loss_pos_y',
 'loss_pos_z',
 'loss_n_photons']

In [9]:
example_set.mc_truth.primary_lepton_1_position_x

<Array [-80.5, -37.8, -85.8, ... -66.7, 58.6] type='10 * float64'>

In [10]:
example_set.event_id

<Array [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] type='10 * int64'>

In [11]:
# The events are split into three parts:
#   - lepton: the light from the generated lepton
#   - hadron: the light produced from the associated hadronic cascade (at production point)
#   - total: the light from both parts
# Each of these fields is formatted the same way
example_set.fields

['event_id', 'mc_truth', 'primary_lepton_1', 'primary_hadron_1', 'total']

In [12]:
# Example for one event
event_id = 0
# The sensors hit
print("Total number of hits:")
print(len(example_set.total.sensor_id[event_id]))
print('The hit sensors: ')
print(example_set.total.sensor_id[event_id])
print('Their associated strings')
print(example_set.total.sensor_string_id[event_id])
print('The hit times')
print(example_set.total.t[event_id])

Total number of hits:
8861
The hit sensors: 
[6, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, ... 51, 51, 52, 52, 52, 52, 52, 52, 52, 53, 54]
Their associated strings
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
The hit times
[1.35e+03, 872, 873, 870, 871, 872, 872, ... 498, 519, 491, 471, 589, 564, 1.13e+03]


In [13]:
def paqruet_plotting(
        data, e_id,
        brightest_event=False,
        channel='total',
        show_lepton=True,
        scaling=20
    ):
    try:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        hit_counts = []
        if brightest_event:
            for event in data[channel].sensor_pos_x:
                hit_counts.append(len(event))
            hit_counts = np.array(hit_counts)
            event_id = np.argmax(hit_counts)
            print('The brightest event has the id %d' % event_id)
            print('The energy of the injected particle is %.1f' % data.mc_truth.injection_energy[event_id])
        else:
            event_id = e_id
        sensor_comb = np.array([
            [data[channel].sensor_pos_x[event_id][i],
            data[channel].sensor_pos_y[event_id][i],
            data[channel].sensor_pos_z[event_id][i]
            ]
            for i in range(len(data[channel].sensor_pos_x[event_id]))
        ])
        hit_doms, hit_counts = np.unique(sensor_comb, axis=0, return_counts=True)
        ax.scatter(
            hit_doms[:, 0],
            hit_doms[:, 1],
            hit_doms[:, 2],
            s=(
                hit_counts / max(hit_counts) * scaling
            ),
            color='r', alpha=0.5)
        if show_lepton:
            ax.scatter(
                data.primary_lepton_1.loss_pos_x[event_id],
                data.primary_lepton_1.loss_pos_y[event_id],
                data.primary_lepton_1.loss_pos_z[event_id],
                s=(
                    data.primary_lepton_1.loss_n_photons[event_id] /
                    max(data.primary_lepton_1.loss_n_photons[event_id]) *
                    scaling
                ),
                color='blue', alpha=0.5)
        # ax.set_xlim(min(hit_doms[:, 0]), max(hit_doms[:, 0]))
        # ax.set_ylim(min(hit_doms[:, 1]), max(hit_doms[:, 1]))
        # ax.set_zlim(min(hit_doms[:, 2]), max(hit_doms[:, 2]))
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_zlabel('Z [m]')
        plt.title(
            'Event %d with pdg id = %d and energy %.1f GeV' % (
                event_id,
                data.mc_truth.injection_type[event_id],
                data.mc_truth.injection_energy[event_id]
                )
        )
    except:
        print('No hits found!')

In [14]:
paqruet_plotting(
    example_set,
    e_id=4,
    brightest_event=True,
    channel='total',
    show_lepton=True,
    scaling=100
)

The brightest event has the id 0
The energy of the injected particle is 10000.1
